## 🔐 API Key Required

Before calling any endpoint, make sure you have a valid **API Key**.

You can generate one here:
👉 https://merchant-stag.transferbankpay.com.br/api-keys

Keep your API Key **secret** and use it only on back end — do not hardcode it in source files. You can easily disable or delete it on the same link.

To create an API Key, you need to provide:

**a) A name**:
This name will appear in the platform history for any payments processed.   It cannot be changed later. We suggest calling it with a hint from where you are using it, e.g., checkout, sales page, etc. You can only use at most 24 alphanumeric characters and spaces.

**b) (optional) An expiration date**:
If you don't provide one, the key will never expire.

---

## 📚 API Documentation

Full API reference is available at:
👉 https://api-docs.transferbankpay.com.br

---

In [6]:
!pip install requests

import requests
import uuid
from pprint import pprint

zsh:1: command not found: pip


In [4]:
API_KEY = 'sYmjFL+e6sJtFQIDISUUmO0sa0ux4vSpDcCYBPqC9bhbRsd3PJCqrgTxpvx2FMe+JcMwUqEvsppzj65v9pIKnWlR1rJlwrJP3YBt5xcgL/Io7A2cNDSgoOJCKG5DgW9w'  # TODO
BASE_URL = 'https://ms-stag.transferbankpay.com.br/integrations'

## Success and Error Responses

Every response has a standard return format, following conventions from Problem Detail RFC (https://datatracker.ietf.org/doc/html/rfc7807):

a) Success:
  - type: URN success identifier.
  - instance: URN identifier which helps to identify the request, for debugging purposes.
  - status: HTTP status code for this response.
  - payload: The most important part – returns specific data related to the request.

b) Error:
  - type: URN identifier which helps to identify the request, for debugging purposes.
  - instance: URN identifier which helps to identify the request, for debugging purposes.
  - status: HTTP status code for this response.
  - title (optional): Human readable title.
  - detail (optional): Human readable description.
  - more (optional): Additional error context attributes

In [7]:
# Get API Key info. Use it to know, for instance, the merchant_id and its available permissions

response = requests.get(
    f'{BASE_URL}/info',
    headers={
        'X-API-KEY': API_KEY
    }
)

"""
Returns:
 - merchant_id: ID of the merchant who owns this API Key
 - api_key: API key identifier
 - name: User given nickname for the API key
 - checkout_enabled: Merchant is allowed to generate checkouts within checkout UI
 - boleto_enabled: Whether this API Key allows creating Boleto payments
 - credit_card_enabled: Whether this API Key allows creating Credit Card payments
"""
print(response.status_code)
pprint(response.json())

merchant_id = None
if response.status_code == 200:
    merchant_id = response.json()['payload']['merchant_id']

200
{'instance': 'urn:tp:instance:trace_id:1-693b9899-209bc1e94e25b60a3021c680:request_id:ZRRJVULA4MLGZYLMQ5',
 'payload': {'api_key': 'OBkUl8Wh',
             'boleto_enabled': True,
             'checkout_enabled': True,
             'credit_card_enabled': False,
             'merchant_id': '66623da7-f1bf-4d3b-ac99-953e8bcd4ac2',
             'name': 'Rafagan teste 2'},
 'status': 200,
 'type': 'urn:tp:integration:ok'}


In [ ]:
# Check the payment status

# Provided by you when a charge is created, or on its return
payment_id = '<uuid>'

response = requests.get(
    f'{BASE_URL}/payments/{payment_id}/status',
    headers={
        'X-API-KEY': API_KEY
    }
)

"""
Returns:
 - status: the charge was paid, failed or is still pending. Possible values:
   - waiting_payment: Nothing happened yet
   - paid: The payment was successful
   - client_paid: Paid, but waiting for settlement (compensation)
   - refund: The charge was refunded
   - chargeback: The payment is in a credit card chargeback dispute
   - med: The payment is in a pix chargeback dispute
   - cancelled: The boleto expired (default is 60 days)
"""
print(response.status_code)
print(response.json())

In [ ]:
# Create pix charge

payload = {
    'payment_id': str(uuid.uuid4()),  # Optional: Use it to correlate with your system. Must be UUID4.
    'customer_id': '<cpf|cnpj>',      # Required: A valid CPF or CNPJ with or without punctuation
    'customer_name': 'John Doe',      # Required: At least two words and 3-character length
    'final_price': '10000E-2'         # Required: Exp notation, always E-2. Example: 10000E-2 = 100.00.
}

response = requests.post(
    f'{BASE_URL}/payments/pix',
    headers={
        'X-API-KEY': API_KEY,
        'Content-Type': 'application/json'
    },
    json=payload
)

"""
Returns:
 - payment_id: If not provided, a new UUID4 will be generated
 - qrcode: The image
 - qrcode_type: The image can be an `url` or `base64`, you need to handle both cases
 - copy&paste: Raw pix text to copy & paste or generate the qrcode yourself
"""
print(response.status_code)
print(response.json())